In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import csv

In [2]:
df = pd.read_csv('Charades_vu17_train.csv')

In [3]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [4]:
print(df.shape)
df = df[pd.notnull(df['actions'])] # drop NA in column['actions']
print(df.shape)

# df = df[df.quality >=6 ]
# df = df[df.relevance == 7] #  和 script 的相關性，但我們不需要管 script ，所以用不到
df['origin_index'] = df.index.values
df.head()

(7985, 11)
(7811, 11)


,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,origin_index
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83,0
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33,1
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33,3
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38,4
5,S6MPZ,EA2K,Home Office / Study (A room in a house used fo...,6.0,6.0,Yes,A person is eating at the desk and lying the p...,chair;desk;food;phone;plate;snacks;table,A person in a blue shirt is eating snacks off ...,c009 0.00 4.30;c011 0.00 39.00;c015 0.00 39.00...,38.46,5


In [5]:
df = df.reset_index(drop=True)
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,origin_index
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83,0
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33,1
2,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33,3
3,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38,4
4,S6MPZ,EA2K,Home Office / Study (A room in a house used fo...,6.0,6.0,Yes,A person is eating at the desk and lying the p...,chair;desk;food;phone;plate;snacks;table,A person in a blue shirt is eating snacks off ...,c009 0.00 4.30;c011 0.00 39.00;c015 0.00 39.00...,38.46,5


In [6]:
dat_id = df['id']
dat_actions = df['actions']
dat_origin_index = df['origin_index']
dat = pd.concat([dat_id, dat_origin_index, dat_actions], axis=1)
split_arr = dat['actions'].str.split(';')

In [7]:
large_arr = []
for element in enumerate(split_arr):
    arr = []
    time = []
    for i in element[1]:
        k = i.split(" ")
        d = k[0]
        t = k[1]
        arr.append(d)
        time.append(t)
#     print(arr)
#     print(time)
    arr = [x for _,x in sorted(zip(time,arr))]
#     print(arr)
    large_arr.append(arr)
dat['split'] = large_arr

In [8]:
dat = dat.drop('actions', 1)

In [9]:
str_actions = dat['split']
int_actions = []
for action in str_actions:
    arr = []
    for act in action:
        a = int(act[1:])
        arr.append(a)
    int_actions.append(arr)

In [10]:
dat
# str_actions

,id,origin_index,split
0,46GP8,0,"[c147, c092]"
1,N11GT,1,"[c075, c127, c153, c098]"
2,KRF68,3,"[c150, c148, c018, c141, c006, c000, c002]"
3,MJO7C,4,"[c015, c107]"
4,S6MPZ,5,"[c009, c011, c015, c017, c019, c059, c061, c06..."
5,7HVU8,6,"[c001, c020, c127, c127, c004, c144, c127]"
6,MCQO5,7,"[c002, c000, c110, c107, c106, c148]"
7,VPIYF,8,[c015]
8,JSUF4,9,"[c102, c127, c151, c123, c106]"
9,8WJIR,10,"[c107, c132, c151, c135, c110, c106, c107, c00..."


In [11]:
import nltk
from nltk.corpus import brown
from nltk import WordNetLemmatizer
from math import log 

# brown = nltk.download('brown')
# wd = nltk.download('wordnet')

wnl=WordNetLemmatizer()

_Fdist = nltk.FreqDist([wnl.lemmatize(w.lower()) for w in brown.words(categories='news')])

_Sents = [[wnl.lemmatize(j.lower()) for j in i] for i in brown.sents(categories='news')]


# int_actions
def p(x):
    return _Fdist[x]/float(len(_Fdist))

def pxy(x,y):
    return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1) / float(len(_Sents) )

def pmi(x,y):
    return  log(pxy(x,y)/(p(x)*p(y)),2) 

In [12]:
_Fdist
# _Sents
# len(_Fdist) # 11892

FreqDist({'the': 6386,
          'fulton': 14,
          'county': 75,
          'grand': 19,
          'jury': 47,
          'said': 406,
          'friday': 41,
          'an': 311,
          'investigation': 12,
          'of': 2861,
          "atlanta's": 4,
          'recent': 20,
          'primary': 17,
          'election': 44,
          'produced': 6,
          '``': 732,
          'no': 121,
          'evidence': 17,
          "''": 702,
          'that': 829,
          'any': 94,
          'irregularity': 3,
          'took': 47,
          'place': 38,
          '.': 4030,
          'further': 18,
          'in': 2020,
          'term-end': 1,
          'presentment': 1,
          'city': 99,
          'executive': 20,
          'committee': 81,
          ',': 5188,
          'which': 245,
          'had': 281,
          'over-all': 2,
          'charge': 29,
          'deserves': 3,
          'praise': 2,
          'and': 2186,
          'thanks': 6,
          'atlanta': 14

In [13]:
p('new')

0.020265724857046755

In [14]:
p('york')

0.004372687521022536

In [15]:
pmi('new','the')

1.8863664858873235

In [16]:
join_str = []
for it in str_actions:
    for s in it:
        join_str.append(s)
        
# join_str

In [17]:
import nltk
from nltk.corpus import brown
from nltk import WordNetLemmatizer
from math import log 

# brown = nltk.download('brown')
# wd = nltk.download('wordnet')

wnl=WordNetLemmatizer()
# lemmatize 詞型還原

_Fdist = nltk.FreqDist([wnl.lemmatize(w.lower()) for w in join_str])

_Sents = [[wnl.lemmatize(j.lower()) for j in i] for i in str_actions]

# int_actions
def p(x):
    return _Fdist[x]/ float(sum(_Fdist.values())) #float(len(_Fdist))

def pxy(x,y):
    return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1) / float(len(_Sents) )

def pmi(x,y):
    return  log(pxy(x,y)/(p(x)*p(y)),2) 

In [18]:
# _Fdist
# _Sents

In [19]:
# c000 Holding some clothes
# c001 Putting clothes somewhere
print(p('c000'))
print(p('c001'))
print(pxy('c000', 'c001'))
print(pmi('c000', 'c001'))

0.012748700034130377
0.01284908349896605
0.04058379208808091
7.952744260921994


In [20]:
# c059 Sitting in a chair
# c011 Sitting at a table
print(p('c059'))
print(p('c011'))
print(pxy('c059', 'c011'))
print(pmi('c059', 'c011'))

0.025296633138589413
0.014615832480073882
0.06734092945845602
7.508862438035805


In [21]:
# c057 Taking off some shoes
# c065 Eating a sandwich
print(p('c057'))
print(p('c065'))
print(pxy('c057', 'c065'))
print(pmi('c057', 'c065'))

0.003935031821558353
0.009215202071914714
0.00038407374215849446
3.404848420521618


In [49]:
classes_raw = open('Charades_v1_classes.txt', 'r')
classes = []
for i in classes_raw.readlines():
    arr = i.split(' ', 1)
    classes.append(arr)
# classes[0][0]
# len(classes) # 157x2, c000 to c156

157

In [69]:
two = []
two_pmi = []
for i in range(0, 157):
    for j in range(i+1, 157):
        arr = [classes[i], classes[j]]
        two.append(arr)
        two_pmi.append(pmi(classes[i][0], classes[j][0]))

In [70]:
two = [x for _,x in sorted(zip(two_pmi,two), reverse=True)]
two_pmi = sorted(two_pmi, reverse=True)

In [126]:
two = np.array(two)
print(two.shape)
two_pmi = np.array(two_pmi)
print(two_pmi.shape)
tmp = two.reshape(-1, 4)
print(tmp.shape)
mypmi = np.column_stack((tmp, two_pmi))

(12246, 2, 2)
(12246,)
(12246, 4)


In [141]:
pd.DataFrame(mypmi, columns={"1st class":0,"1st class name":1,"2nd class":2,"2nd class name":3,"pointwise mutual info":4})

,1st class,1st class name,2nd class,2nd class name,pointwise mutual info
0,c007,Fixing a door\n,c140,Fixing a doorknob\n,11.737787296221445
1,c060,Standing on a chair\n,c103,Fixing a light\n,11.583459149830153
2,c099,Putting a broom somewhere\n,c101,Throwing a broom somewhere\n,10.896124323388547
3,c084,Holding a picture\n,c086,Putting a picture somewhere\n,10.72689898007871
4,c083,Reaching for and grabbing a picture\n,c086,Putting a picture somewhere\n,10.636858387370665
5,c083,Reaching for and grabbing a picture\n,c084,Holding a picture\n,10.597172806920494
6,c111,Washing a cup/glass/bottle\n,c121,Wash a dish/dishes\n,10.439843841877822
7,c084,Holding a picture\n,c085,Laughing at a picture\n,10.414308749770683
8,c089,Closing a window\n,c090,Opening a window\n,10.354009670361135
9,c012,Tidying up a table\n,c013,Washing a table\n,10.270811321157376


In [142]:
_Fdist

FreqDist({'c000': 635,
          'c001': 640,
          'c002': 479,
          'c003': 303,
          'c004': 266,
          'c005': 81,
          'c006': 618,
          'c007': 27,
          'c008': 784,
          'c009': 879,
          'c010': 46,
          'c011': 728,
          'c012': 202,
          'c013': 42,
          'c014': 241,
          'c015': 1055,
          'c016': 673,
          'c017': 219,
          'c018': 298,
          'c019': 217,
          'c020': 678,
          'c021': 356,
          'c022': 280,
          'c023': 278,
          'c024': 89,
          'c025': 187,
          'c026': 615,
          'c027': 280,
          'c028': 248,
          'c029': 92,
          'c030': 257,
          'c031': 58,
          'c032': 483,
          'c033': 575,
          'c034': 276,
          'c035': 286,
          'c036': 154,
          'c037': 125,
          'c038': 232,
          'c039': 96,
          'c040': 331,
          'c041': 181,
          'c042': 201,
          'c043': 

In [143]:
_Sents

[['c147', 'c092'],
 ['c075', 'c127', 'c153', 'c098'],
 ['c150', 'c148', 'c018', 'c141', 'c006', 'c000', 'c002'],
 ['c015', 'c107'],
 ['c009',
  'c011',
  'c015',
  'c017',
  'c019',
  'c059',
  'c061',
  'c063',
  'c156',
  'c061'],
 ['c001', 'c020', 'c127', 'c127', 'c004', 'c144', 'c127'],
 ['c002', 'c000', 'c110', 'c107', 'c106', 'c148'],
 ['c015'],
 ['c102', 'c127', 'c151', 'c123', 'c106'],
 ['c107',
  'c132',
  'c151',
  'c135',
  'c110',
  'c106',
  'c107',
  'c009',
  'c131',
  'c152',
  'c109',
  'c154',
  'c006'],
 ['c004', 'c075', 'c107', 'c152', 'c135', 'c151', 'c106', 'c118'],
 ['c001', 'c002', 'c034', 'c148', 'c129', 'c155', 'c096', 'c129', 'c106'],
 ['c011', 'c014', 'c059'],
 ['c124', 'c107', 'c106', 'c154', 'c146'],
 ['c107', 'c118', 'c109', 'c119', 'c009', 'c088'],
 ['c155',
  'c023',
  'c063',
  'c061',
  'c063',
  'c081',
  'c097',
  'c020',
  'c006',
  'c141',
  'c000',
  'c001'],
 ['c012', 'c154'],
 ['c008', 'c097', 'c104', 'c130', 'c118', 'c061', 'c063'],
 ['c101', 